# Data Visualization in Python II
In this notebook we will learn how to create visualizations using [__plotly__](https://plotly.com/python/). The plotly Python library is an interactive, open-source plotting library that supports over many chart types covering a wide range of statistical, financial, geographic, scientific, and 3-dimensional use-cases. We will be exploring the advanced functionality in plotly, as map rendering and interactive features.

In [ ]:
!pip install plotly==4.9.0

## Maps

Plotly allows the creation of interactive maps. Here we load a [GeoJSON](https://geojson.org/) file which the geometry information for US counties.

In [ ]:
from urllib.request import urlopen
import json

# Load GeoJSON
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In this map data structure, the `features.id` field stores the reference [FIPS code](https://en.wikipedia.org/wiki/FIPS_county_code) of the county. 

In [ ]:
# Show county 0 features
counties["features"][0]

Now, let's load unemployment data by county, which is also indexed by FIPS code.

In [ ]:
# Unemployment data by county as a dataframe
import pandas as pd
df_unemployment = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",dtype={"fips": str})
df_unemployment.head()

There are several ways to create maps combining this information un plotly, we will be using [`plotly.express`](https://plotly.com/python/plotly-express/), an easy-to-use, high-level interface to Plotly, which operates on a variety of types of data and produces easy-to-style figures. In particular, we are using [`choropleth_mapbox`](https://plotly.com/python/mapbox-county-choropleth/).

In [ ]:
import plotly.express as px

# The data to be shown (df_unemployment), the map (counties), the matching key (fips) and the variable (unemp) 
fig = px.choropleth_mapbox(df_unemployment, geojson=counties, locations='fips', color='unemp',
                           color_continuous_scale="Inferno",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'unemp':'unemployment rate'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}) # margins
fig.show()

If the GeoJSON you are using either does not have an id field or you wish you use one of the keys in the properties field, you may use the `featureidkey` parameter to specify where to match the values of `locations`. In the following GeoJSON object/data-file pairing, the values of `properties.distric` match the values of the district column:

In [ ]:
df_elections = px.data.election() # Load example dataframe
geojson      = px.data.election_geojson() # Load example geojson

# Matching keys
print(df_elections["district"][2])
print(geojson["features"][0]["properties"])

In [ ]:
fig = px.choropleth_mapbox(df_elections, geojson=geojson, color="Bergeron",
                           locations="district", featureidkey="properties.district", # specify matching columns
                           center={"lat": 45.5517, "lon": -73.7073},
                           mapbox_style="carto-positron", zoom=9)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

If instead of using a continuous variable as `Bergeron` to colorize the map we select a non-numerical column like `winner`, `choropleth_mapbox` will automatically show a discret scale and the legend. 

In [ ]:
fig = px.choropleth_mapbox(df_elections, geojson=geojson, color="winner",  # discrete variable
                           locations="district", featureidkey="properties.district",
                           center={"lat": 45.5517, "lon": -73.7073},
                           mapbox_style="carto-positron", zoom=9)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

It is also possible to show markers on top of a map using [`scatter_mapbox`](https://plotly.com/python/scattermapbox/):

In [ ]:
df = px.data.carshare()

fig = px.scatter_mapbox(df, lat="centroid_lat", lon="centroid_lon", color="peak_hour", size="car_hours",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10,
                  mapbox_style="carto-positron")

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()


## Interaction
Let's go through a series of interactive examples. 

### Buttons
You can find more examples on using buttons [here](https://plotly.com/python/custom-buttons/).

In [ ]:
import plotly.graph_objects as go

# Load dataset
df_volcano = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/volcano.csv")
df_volcano

Create figure to visualize data as a 3D surface or as a 2D heatmap. 

In [ ]:
# Create figure
fig = go.Figure()

# Add surface trace
fig.add_trace(go.Surface(z=df_volcano.values.tolist(), colorscale="Inferno"))

# Custom plot sizing
fig.update_layout(
    width=800,
    height=900,
    autosize=False,
    margin=dict(t=0, b=0, l=0, r=0),
    template="plotly_white", 
)

# Update 3D scene options
fig.update_scenes(
    aspectratio=dict(x=1, y=1, z=0.7),
    aspectmode="manual"
)
    
# Add simple menu with 2 buttons  
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=["type", "surface"],
                    label="3D Surface",
                    method="restyle"
                ),
                dict(
                    args=["type", "heatmap"],
                    label="Heatmap",
                    method="restyle"
                )
            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.11,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

# Add annotation to buttons
fig.update_layout(
    annotations=[
        dict(text="Trace type:", showarrow=False, x=0, y=1.08, yref="paper", align="left")
    ]
)

fig.show()

### Slider
You can find more examples on using sliders [here](https://plotly.com/python/sliders/).

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Create figure
fig = go.Figure()

# Add traces, one for each slider step
for step in np.arange(0, 5, 0.1):
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="#00CED1", width=6),
            name="𝜈 = " + str(step),
            x=np.arange(0, 10, 0.01),
            y=np.sin(step * np.arange(0, 10, 0.01))))

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

Plotly Express also provide sliders, but with implicit animation using the "animate" method described above. The animation play button can be ommited by removing updatemenus in the layout:

In [ ]:
import plotly.express as px

df = px.data.gapminder()
fig = px.scatter(df, x="gdpPercap", y="lifeExp", animation_frame="year", animation_group="country",
           size="pop", color="continent", hover_name="country",
           log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])

fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.show()

### Range Slider
You can find more examples on using range sliders [here](https://plotly.com/python/range-slider/).

In [ ]:
# Load data
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv")
df.columns = [col.replace("AAPL.", "") for col in df.columns]

# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(df.Date), y=list(df.High)))

# Set title
fig.update_layout(
    title_text="Time series with range slider and selectors"
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([ # Predefined ranges for the buttons
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()